This Notebook connects to a mongodb database, connects to the Lakeshore temperature monitors that we are using, creates a data object by scanning these instruments, and inserts them into the MongoDB database.

In [1]:
import time
import numpy as np
from datetime import datetime
from calendar import timegm
from pymongo import MongoClient, ASCENDING, DESCENDING
import pyvisa
import serial

# Setup and configure Mongo

In [2]:
def dbConnect():
    client = MongoClient('mongodb://localhost/')
    db = client.data
    collection = db.adr2datas
    db.collection_names()
    infocollection = db.adr2infos
    return collection, infocollection
collection,infocollection = dbConnect()

In [3]:
collection.name

u'adr2datas'

In [4]:
infocollection.full_name

u'data.adr2infos'

# Add Single Data file to DB

### Returns dict with random data

In [3]:
def rand_data():
    post = {
        "timeStamp": timegm(datetime.now().timetuple()),
        "baseTemp": .05*np.random.rand()+.05,
        "threeKTemp": 0.7*np.random.rand()+2.8,
        "sixtyKTemp": np.random.rand()+60,
        "magnetVoltage": np.random.rand(),
        "psVoltage": np.random.rand(),
        "psCurrent": np.random.rand(),
        "currentJob": "None",
        "percentComplete": 0,
        "switchState": "Closed"
    }
    return post
rand_data()

{'baseTemp': 0.07545333274376156,
 'currentJob': 'None',
 'magnetVoltage': 0.861983735742138,
 'percentComplete': 0,
 'psCurrent': 0.6623301643731142,
 'psVoltage': 0.9200453201886846,
 'sixtyKTemp': 60.51191939807759,
 'switchState': 'Closed',
 'threeKTemp': 3.468096524774105,
 'timeStamp': 1440695249}

In [8]:
for data in collection.find().limit(10):
    print(data)

## Get Data from Instruments

### Create Instrument Objects

In [3]:
def connectInstruments():
    rm = pyvisa.ResourceManager()
    rm.list_resources()
    inst1 = rm.open_resource('GPIB0::1::INSTR', read_termination = '\x00')
    inst2 = rm.open_resource('GPIB0::2::INSTR', read_termination = '\x00')
    inst3 = rm.open_resource('GPIB0::6::INSTR', read_termination = '\x00')
    return [inst1,inst2,inst3]

In [4]:
x = connectInstruments()
x[0].ask('*IDN?')

C:\Anaconda\lib\site-packages\pyvisa\resources\messagebased.py:387: UserWarning: read string doesn't end with termination characters
  return self.read()


u'LSCI,MODEL340,341911,042304'

In [5]:
rm = pyvisa.ResourceManager()
rm.list_resources()

(u'ASRL1::INSTR', u'GPIB0::1::INSTR', u'GPIB0::2::INSTR', u'GPIB0::6::INSTR')

### Get Data Object

In [6]:
# inst = connectInstruments()
# for i in inst:
#     i.encoding = 'utf-8'
#     print(i.encoding)
    
def get_data(inst):
    post = {
        "timeStamp": timegm(datetime.now().timetuple()),
        "baseTemp": float(inst[1].ask('KRDG? B')),
        "oneKTemp": float(inst[0].ask('KRDG? C')),
        "threeKTemp": float(inst[0].ask('KRDG? A')),
        "sixtyKTemp": float(inst[0].ask('KRDG? B')),
        "magnetVoltage": float(inst[0].ask('SRDG? D')),
        "psVoltage": 0,
        "psCurrent": 0,
        "currentJob": "None",
        "percentComplete": 0,
        "switchState": "Closed"
    }
    return post
get_data(connectInstruments())

{'baseTemp': 0.0,
 'currentJob': 'None',
 'magnetVoltage': -0.0084,
 'oneKTemp': 251.835,
 'percentComplete': 0,
 'psCurrent': 0,
 'psVoltage': 0,
 'sixtyKTemp': 223.319,
 'switchState': 'Closed',
 'threeKTemp': 278.74,
 'timeStamp': 1441905513}

In [7]:
float(inst[1].ask('KRDG? B'))*1000

NameError: name 'inst' is not defined

In [67]:
'abb0b7b8aeb1b3b6c5abb0'.decode("hex")

'\xab\xb0\xb7\xb8\xae\xb1\xb3\xb6\xc5\xab\xb0'

###Validate Data, Determine Fridge Status

In [ ]:
def fridgeStatus(data, infocollection):
    if (data['sixtyKTemp'] >= 289.9):
        infocollection.

### Update every 5 seconds

In [8]:
inst = connectInstruments()
while True:
#     temp = rand_data()
#     collection.insert(temp)
    try:
        temp = get_data(inst)

        
    except UnicodeDecodeError:
        print("Read error at " + datetime.now().strftime('%H:%M:%S'))
        inst = connectInstruments()
        
    else:
        if (temp['sixtyKTemp'] <= 300):
            try:
                collection.insert(temp)
            except:
                print("Database error at " + datetime.now().strftime('%H:%M:%S'))
                
    time.sleep(5)

Database error at 17:26:19
Database error at 17:26:35
Database error at 17:28:48


KeyboardInterrupt: 

In [ ]:
while True:
    temp = get_data()
    if (temp['sixtyKTemp'] <= 290):
            collection.insert(temp)
    time.sleep(5)

In [10]:
temp['sixtyKTemp']

296.278

##Log File

In [ ]:
logFileName = 'log.txt'

##Scrape Data + Magnet Control

In [ ]:
while True:
    try:
        temp = get_data(inst)
        
    except UnicodeDecodeError:
        print("Read error at " + datetime.now().strftime('%H:%M:%S'))
        inst = connectInstruments()
        
    else:
        if (temp['sixtyKTemp'] <= 290):
            try:
                collection.insert(temp)
            except:
                print("Database error at " + datetime.now().strftime('%H:%M:%S'))
    
                
    time.sleep(5)